In [27]:
#! pip install prophet
#! pip install pystan==2.19.1.1
#! pip install prophet --no-cache-dir --force-reinstall
import pandas as pd
import numpy as np
import warnings
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
warnings.filterwarnings("ignore")
# STEP 1: Import Libraries


from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from joblib import Parallel, delayed


In [28]:
df = pd.read_csv('../data/cleaned_data.csv', parse_dates=['date'])
df.head()

,date,store_nbr,family,sales,onpromotion,holiday_type,locale,transferred,dcoilwtico,city,state,store_type,cluster,transactions,year,month,week,quarter,day_of_week,is_crisis,sales_lag_7,rolling_mean_7,is_weekend,is_holiday,promo_last_7_days,days_to_holiday,promotion_status
0,2013-01-01,1,AUTOMOTIVE,0.00,0,Holiday,National,False,93.14,Quito,Pichincha,D,13,0.00,2013,1,1,1,Tuesday,0,0.00,0.00,0,1,0.00,0,Not On Promotion
1,2013-01-01,1,BABY CARE,0.00,0,Holiday,National,False,93.14,Quito,Pichincha,D,13,0.00,2013,1,1,1,Tuesday,0,0.00,0.00,0,1,0.00,0,Not On Promotion
2,2013-01-01,1,BEAUTY,0.00,0,Holiday,National,False,93.14,Quito,Pichincha,D,13,0.00,2013,1,1,1,Tuesday,0,0.00,0.00,0,1,0.00,0,Not On Promotion
3,2013-01-01,1,BEVERAGES,0.00,0,Holiday,National,False,93.14,Quito,Pichincha,D,13,0.00,2013,1,1,1,Tuesday,0,0.00,0.00,0,1,0.00,0,Not On Promotion
4,2013-01-01,1,BOOKS,0.00,0,Holiday,National,False,93.14,Quito,Pichincha,D,13,0.00,2013,1,1,1,Tuesday,0,0.00,0.00,0,1,0.00,0,Not On Promotion


Load & Prepare Data

In [29]:
df['ds'] = pd.to_datetime(df['date'])
df['y'] = df['sales']

In [30]:
df['y'] = np.log1p(df['y'])  # log(y + 1) to avoid issues with zero


In [31]:
from sklearn.preprocessing import RobustScaler
import numpy as np

# STEP 1: Preprocessing BEFORE calling any training function

# 1. Log-transform the target variable
df['y'] = np.log1p(df['y'])

# 2. List of regressors used in training
regressors = ['onpromotion', 'is_weekend', 'is_holiday', 
              'promo_last_7_days', 'sales_lag_7', 'rolling_mean_7']

# 3. Apply RobustScaler per family
scaler = RobustScaler()
df[regressors] = df.groupby('family')[regressors].transform(
    lambda x: scaler.fit_transform(x.values.reshape(-1, 1)).flatten()
)

# STEP 2: Now call your training function
# Example: results = train_model_for_family('BEVERAGES', df)


 Select Relevant Columns

In [32]:
columns_to_keep = [
    'ds', 'y', 'family', 'onpromotion', 'is_weekend',
    'is_holiday', 'promo_last_7_days', 'sales_lag_7', 'rolling_mean_7'
]

df = df[columns_to_keep]

# Drop missing values or fill
df.dropna(inplace=True)



In [33]:
# STEP 2: Define Evaluation Function
def evaluate_forecast(y_true, y_pred):
    """
    Calculate MAE, MSE, RMSE, and R²
    """
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    return mae, mse, rmse, r2


In [34]:
def train_model_for_family(fam, df):
    """
    Train Prophet model for each family and evaluate the performance.
    Assumes y has already been log1p-transformed.
    """
    df_fam = df[df['family'] == fam].copy()
    df_fam.drop('family', axis=1, inplace=True)

    # Split data into train and test
    cutoff = pd.to_datetime('2017-01-01')  # fixed date split
    train = df_fam[df_fam['ds'] < cutoff]
    test = df_fam[df_fam['ds'] >= cutoff]

    if len(test) < 10 or len(train) < 100:  # Skip short series
        return None  # Skip this family if there's not enough data

    # Build Prophet model with optimized configuration
    model = Prophet(n_changepoints=10, seasonality_mode='multiplicative', yearly_seasonality=True, daily_seasonality=False)
    
    # Adding regressors
    regressors = ['onpromotion', 'is_weekend', 'is_holiday', 'promo_last_7_days', 'sales_lag_7', 'rolling_mean_7']
    for reg in regressors:
        model.add_regressor(reg)
    
    # Fit the model
    model.fit(train)

    # Forecasting
    future = test[['ds'] + regressors]
    forecast = model.predict(future)

    # Inverse log1p transformation of predictions and true values
    y_pred = np.expm1(forecast['yhat'].values)
    y_true = np.expm1(test['y'].values)

    # Evaluate performance
    mae, mse, rmse, r2 = evaluate_forecast(y_true, y_pred)

    return {'family': fam, 'mae': mae, 'mse': mse, 'rmse': rmse, 'r2': r2}


In [35]:
# STEP 4: Parallelize the Training for Each Family
def train_prophet_per_family(df):
    """
    Train Prophet models for each family in parallel and return evaluation results.
    """
    families = df['family'].unique()
    
    # Parallelizing model training for each family using all available CPU cores
    results = Parallel(n_jobs=-1)(delayed(train_model_for_family)(fam, df) for fam in families)
    
    # Filter out any 'None' results (families that were skipped due to insufficient data)
    results = [result for result in results if result is not None]
    
    # Convert results into a DataFrame
    return pd.DataFrame(results)


In [36]:
# STEP 5: Run the Function and View Results
# Assuming df is already loaded and preprocessed, now let's run the model
results_df = train_prophet_per_family(df)

# Display the results sorted by R² (best model on top)
results_df.sort_values(by='r2', ascending=False, inplace=True)
print(results_df)


                        family      mae              mse       rmse  \
31  SCHOOL AND OFFICE SUPPLIES     0.73             1.27       1.13   
1                    BABY CARE     0.17             0.11       0.32   
23                   MAGAZINES     0.80             0.97       0.98   
14                    HARDWARE     0.53             0.40       0.63   
21                    LINGERIE     0.67             0.84       0.91   
17             HOME APPLIANCES     0.32             0.25       0.50   
0                   AUTOMOTIVE     0.60             0.61       0.78   
16         HOME AND KITCHEN II     0.76             0.96       0.98   
32                     SEAFOOD     1.02             2.40       1.55   
26                PET SUPPLIES     1.11             1.85       1.36   
19                  LADIESWEAR     1.12             3.52       1.88   
29              PREPARED FOODS     1.06             1.81       1.35   
22            LIQUOR,WINE,BEER     1.44             3.27       1.81   
13    

In [ ]:
# Step 1: Get all unique families
families = df['family'].unique()

# Step 2: Train model for each family
results = [train_model_for_family(fam, df) for fam in families]

# Step 3: Filter out None results
results_cleaned = [r for r in results if r is not None]

# Step 4: Create results DataFrame including r2
results_df = pd.DataFrame(results_cleaned)

# Step 5: Display performance
print(results_df[['family', 'mae', 'mse', 'rmse', 'r2']])


21:09:58 - cmdstanpy - INFO - Chain [1] start processing
21:10:10 - cmdstanpy - INFO - Chain [1] done processing
21:10:23 - cmdstanpy - INFO - Chain [1] start processing
21:11:10 - cmdstanpy - INFO - Chain [1] done processing
21:11:23 - cmdstanpy - INFO - Chain [1] start processing
21:11:40 - cmdstanpy - INFO - Chain [1] done processing
21:11:52 - cmdstanpy - INFO - Chain [1] start processing
21:12:07 - cmdstanpy - INFO - Chain [1] done processing
21:12:19 - cmdstanpy - INFO - Chain [1] start processing
21:13:19 - cmdstanpy - INFO - Chain [1] done processing
21:13:31 - cmdstanpy - INFO - Chain [1] start processing
21:14:01 - cmdstanpy - INFO - Chain [1] done processing
21:14:14 - cmdstanpy - INFO - Chain [1] start processing
21:16:45 - cmdstanpy - INFO - Chain [1] done processing
21:16:58 - cmdstanpy - INFO - Chain [1] start processing
21:17:21 - cmdstanpy - INFO - Chain [1] done processing
21:17:33 - cmdstanpy - INFO - Chain [1] start processing
21:18:41 - cmdstanpy - INFO - Chain [1]